---

💡 **NOTA**: Necesitaremos usar una GPU para ejecutar los ejemplos en este cuaderno. En Google Colab, andá a
**Runtime > Change runtime type > Hardware accelerator > GPU > GPU type > T4**.

---

In [11]:
%%capture
!pip install --upgrade pip
!pip uninstall -y transformers accelerate
!pip install transformers==4.35.2 accelerate==0.25.0
!pip install sentence-transformers==3.0.1 gensim==4.3.2 scikit-learn==1.5.0 numpy==1.26.0 scipy==1.10.0

# Descargando y Ejecutando un LLM

El primer paso es cargar nuestro modelo en la GPU para una inferencia más rápida. Tené en cuenta que cargamos el modelo y el tokenizador por separado para poder explorarlos individualmente.

In [14]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Cargar modelo y tokenizador
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cpu",
    torch_dtype="auto",
    trust_remote_code=False,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Escribí un email disculpándote con María por el incidente en el asado del domingo. Explicá cómo sucedió.<|assistant|> Estimada María,


Espero que este mensaje te encuentre bien.


In [ ]:
# Ejemplo en español
prompt = "Escribí un email disculpándote con María por el incidente en el asado del domingo. Explicá cómo sucedió.<|assistant|>"

# Tokenizar el prompt de entrada
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cpu")


In [ ]:
# Generar el texto
generation_output = model.generate(
  input_ids=input_ids,
  max_new_tokens=200
)

# Imprimir la salida
print(tokenizer.decode(generation_output[0]))

In [ ]:
# Generar el texto
generation_output = model.generate(
  input_ids=input_ids,
  max_new_tokens=100
)

# Imprimir la salida
print(tokenizer.decode(generation_output[0]))

# Comparando Tokenizadores de LLMs Entrenados

En esta sección, vamos a ver cómo diferentes modelos tokenizan el texto. Es interesante ver cómo cada modelo maneja el español, los emojis y diferentes tipos de texto.

In [3]:
colors_list = [
    '102;194;165', '252;141;98', '141;160;203',
    '231;138;195', '166;216;84', '255;217;47'
]

def show_tokens(sentence, tokenizer_name):
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    token_ids = tokenizer(sentence).input_ids
    for idx, t in enumerate(token_ids):
        print(
            f'\x1b[0;30;48;2;{colors_list[idx % len(colors_list)]}m' +
            tokenizer.decode(t) +
            '\x1b[0m',
            end=' '
        )

In [4]:
# Texto de ejemplo adaptado al español y contexto argentino
text = """
Español y MAYÚSCULAS
🧉 ñ
show_tokens False None elif == >= else: dos tabs:"    " Tres tabs: "       "
12.0*50=600
El mate está muy caliente
"""

In [5]:
# Probamos diferentes tokenizadores
show_tokens(text, "bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Exception ignored on calling ctypes callback function: <function ThreadpoolController._find_libraries_with_dl_iterate_phdr.<locals>.match_library_callback at 0x7aab09f9c180>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/threadpoolctl.py", line 1005, in match_library_callback
    self._make_controller_from_path(filepath)
  File "/usr/local/lib/python3.11/dist-packages/threadpoolctl.py", line 1187, in _make_controller_from_path
    lib_controller = controller_class(
                     ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/threadpoolctl.py", line 114, in __init__
    self.dynlib = ctypes.CDLL(filepath, mode=_RTLD_NOLOAD)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/ctypes/__init__.py", line 376, in __init__
    self._handle = _dlopen(self._name, mode)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^
OSError: dlopen() error


[CLS] es ##pan ##ol y may ##us ##cula ##s [UNK] n show _ token ##s false none eli ##f = = > = else : dos tab ##s : " " tres tab ##s : " " 12 . 0 * 50 = 600 el mate est ##a mu ##y cal ##iente [SEP] 

In [6]:
show_tokens(text, "bert-base-cased")

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

[CLS] E ##sp ##a ##ño ##l y MA ##Y ##Ú ##SC ##U ##LA ##S [UNK] ñ show _ token ##s F ##als ##e None el ##if = = > = else : dos ta ##bs : " " T ##res ta ##bs : " " 12 . 0 * 50 = 600 El mate est ##á m ##uy ca ##lient ##e [SEP] 

In [7]:
show_tokens(text, "microsoft/Phi-3-mini-4k-instruct")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


 
 Esp añ ol y MA Y Ú SC UL AS 
 � � � �  ñ 
 show _ to kens False None elif == >= else : dos tabs :"    " T res tabs : "       " 
 1 2 . 0 * 5 0 = 6 0 0 
 El mate está muy cal iente 
 

In [8]:
show_tokens(text, "gpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]


 E sp a ñ ol  y  MAY � � SC UL AS 
 � � �  � � 
 show _ t ok ens  False  None  el if  ==  >=  else :  dos  tabs :"        "  T res  tabs :  "              " 
 12 . 0 * 50 = 600 
 El  mate  est á  m uy  cal ient e 
 

# Embeddings de Palabras Contextualizados desde un Modelo de Lenguaje (Como BERT)

Los embeddings contextualizados nos permiten obtener representaciones numéricas de palabras que tienen en cuenta el contexto en el que aparecen.

In [9]:
from transformers import AutoModel, AutoTokenizer

# Cargar tokenizador
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-base")

# Cargar modelo de lenguaje
model = AutoModel.from_pretrained("microsoft/deberta-v3-xsmall")

# Tokenizar la oración
tokens = tokenizer('Hola mundo', return_tensors='pt')

# Procesar los tokens
output = model(**tokens)[0]

print("Forma del tensor de salida:")
print(output.shape)

print("\nTokens individuales:")
for token in tokens['input_ids'][0]:
    print(tokenizer.decode(token))

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/474 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/241M [00:00<?, ?B/s]

Forma del tensor de salida:
torch.Size([1, 6, 384])

Tokens individuales:
[CLS]
H
ola
 mund
o
[SEP]


# Embeddings de Texto (Para Oraciones y Documentos Completos)

Los embeddings de texto nos permiten convertir oraciones completas en vectores numéricos, útiles para búsqueda semántica y análisis de similitud.

In [13]:
from sentence_transformers import SentenceTransformer

# Cargar modelo
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

# Convertir texto a embeddings
vector = model.encode("¡La mejor película que vi!")

print("Dimensiones del vector:", vector.shape)

RuntimeError: Failed to import transformers.trainer because of the following error (look up to see its traceback):
cannot import name 'is_torch_less_than_1_11' from 'transformers.pytorch_utils' (/usr/local/lib/python3.11/dist-packages/transformers/pytorch_utils.py)